In [2]:
import pandas as pd
import numpy as np
import nltk

# Baum-Welch Algorithm

In [16]:
alpha = np.zeros( (5, 9) )
alpha

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [18]:
X = np.reshape([i for i in range(9)]*5, (9,5))
X

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 0],
       [1, 2, 3, 4, 5],
       [6, 7, 8, 0, 1],
       [2, 3, 4, 5, 6],
       [7, 8, 0, 1, 2],
       [3, 4, 5, 6, 7],
       [8, 0, 1, 2, 3],
       [4, 5, 6, 7, 8]])

In [21]:
alpha.T[1] = X[1]
alpha

array([[ 0.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [95]:
epsilon = np.random.random( (4, 3, 2) )
epsilon

array([[[ 0.98805003,  0.40372642],
        [ 0.53797724,  0.23448019],
        [ 0.05338353,  0.51783128]],

       [[ 0.86992223,  0.80669244],
        [ 0.65369358,  0.79343884],
        [ 0.6183715 ,  0.17904411]],

       [[ 0.8651411 ,  0.82390794],
        [ 0.32326851,  0.18333333],
        [ 0.24473372,  0.36866599]],

       [[ 0.60980616,  0.14317828],
        [ 0.82471555,  0.64424388],
        [ 0.43957701,  0.51154068]]])

In [96]:
for i in range(len(gamma)):
        sum_gamma = np.sum(gamma[i])
        gamma[i] = np.divide(gamma[i], sum_gamma)
epsilon

array([[[ 0.98805003,  0.40372642],
        [ 0.53797724,  0.23448019],
        [ 0.05338353,  0.51783128]],

       [[ 0.86992223,  0.80669244],
        [ 0.65369358,  0.79343884],
        [ 0.6183715 ,  0.17904411]],

       [[ 0.8651411 ,  0.82390794],
        [ 0.32326851,  0.18333333],
        [ 0.24473372,  0.36866599]],

       [[ 0.60980616,  0.14317828],
        [ 0.82471555,  0.64424388],
        [ 0.43957701,  0.51154068]]])

In [37]:
epsilon.T

array([[[ 0.43373655,  0.48067942,  0.68359986,  0.65446609],
        [ 0.26860673,  0.14580613,  0.91799423,  0.45592848],
        [ 0.54717448,  0.55080247,  0.00952493,  0.55444581]],

       [[ 0.56168666,  0.99377441,  0.93556425,  0.20038123],
        [ 0.34921011,  0.15370216,  0.24626346,  0.86964184],
        [ 0.93727877,  0.25702175,  0.23745332,  0.65777195]]])

In [40]:
epsilon.T[1][0]

array([ 0.56168666,  0.99377441,  0.93556425,  0.20038123])

In [33]:
print(np.shape(epsilon))
print(np.shape(epsilon.T))

(9, 5, 4)
(4, 5, 9)


In [85]:
alpha = np.zeros( (observation_len, latent_len) )
alpha[1]

array([ 0.,  0.])

In [92]:
def calc_forward_messages(unary_matrix, 
                          transition_matrix,
                          emission_matrix,
                          latent_indice_len,
                          observation_sentence):
    """
    Calcualte the forward messages ~ alpha values.
    
    unary_matrix: marginal probability ~ initial matrix.
    return: alpha
    """
    
    # TODO: verify matrix length
    observation_len = len(observation_sentence)
    
    alpha = np.zeros( (observation_len, latent_indice_len) )
    alpha[0] = np.multiply(emission_matrix[0],unary_matrix[0])
    
    for t in np.arange(1, observation_len):
        for j in range(latent_indice_len):
            for i in range(len(latent_indice_map)):
                sum_al += alpha[t-1][i] * transition_matrix[j][i]
            alpha[t][j] = emission_matrix[ observation_sentence[t] ][ j ] * sum_al
    
    return alpha
    
    
def calc_backward_messages(transition_matrix,
                           emission_matrix,
                           latent_indice_len,
                           observation_sentence):
    """
    Calcualte the backward messages ~ beta values.
    
    return: beta
    """
    # TODO: verify matrix length
    observation_len = len(observation_sentence)
    
    beta = np.ones( (observation_len, latent_indice_len) )
    
    for t in np.invert(range(observation_len-1)):
        for i in range(latent_indice_len):
            for j in range(len(latent_indice_map)):
                beta[t][i] += beta[t+1][j] * transition_matrix[j][i] * emission_matrix[ observation_sentence[t+1] ][j]
    
    return beta

def calc_updated_matrix(alpha, beta, latent_indice_len, observation_sentence):
    """
    Calcualte the gama and epsilon values in order to reproduce better transition and emission matrix.
    
    return: unary_matrix, transition_matrix, emission_matrix
    """
    # TODO: verify matrix length
    observation_len = len(observation_sentence)
    
    
    gamma = np.multiply(alpha, beta)
    epsilon = np.zeros( (observation_len-1, latent_indice_len, latent_indice_len) )
    
    for i in range(len(gamma)):
        sum_gamma = np.sum(gamma[i])
        gamma[i] = np.divide(gamma[i], sum_gamma)
    
    for t in range(observation_len - 1):   
        for i in range(latent_len):
            for j in range(latent_len):
                epsilon[t][i][j] = alpha[t][i]*transition_matrix[i][j]*beta[t+1][j]*emission_matrix[t+1][j]
        # Normalization
        sum_ep = np.sum(epsilon[t])
        epsilon[t] = np.divide(epsilon[t], sum_ep)
    
    # Update unary matrix
    unary_matrix = gamma[0]
    
    transition_matrix = np.zeros( (observation_len, latent_len) )
    emission_matrix = np.zeros( (latent_len, latent_len) )
    
    # Update transition matrix
    for i in range(latent_len):
        sum_gamma = np.sum(gamma.T[i])
        for j in range(latent_len):
            sum_ep = np.sum(epsilon.T[j][i])
            transition_matrix[i][j] = sum_ep/sum_gamma
    
    # Update emission matrix
    sum_gamma = [np.sum(gamma.T[i]) for i in range(latent_len)]    
    for i in range(latent_len):
        emission_matrix.T[i] = np.divide(gamma.T[i], sum_gamma[i])
    
    return unary_matrix, transition_matrix, emission_matrix

In [93]:
unary_matrix = [.85, .15]
transition_matrix = [
    [.3, .7], 
    [.1, .9]
]
emission_matrix = [
    [.4, .5],
    [.6, .5]
]
latent_len = 2
observation_len = 4

In [94]:
calc_forward_messages(unary_matrix, transition_matrix, emission_matrix, latent_len, observation_len)

1 0
1 1
2 0


IndexError: list index out of range

# Neural Network

# Backup

In [ ]:
def calc_forward_messages(unary_matrix, transition_matrix, emission_matrix, latent_len, observation_len):
    """
    Calcualte the forward messages ~ alpha values.
    
    unary_matrix: marginal probability ~ initial matrix.
    return: alpha
    """
    
    # TODO: verify matrix length
    alpha = np.zeros( (observation_len, latent_len) )
    
#     for i in range(latent_len):
    alpha[0] = np.multiply(emission_matrix[0],unary_matrix[0])
    for t in np.arange(1, observation_len):
        for j in range(latent_len):
            alpha[t][j] = emission_matrix[t][j] * np.sum( np.multiply(alpha[t-1], transition_matrix[j]) )
    
    return alpha
    
    
def calc_backward_messages(transition_matrix, emission_matrix, latent_len, observation_len):
    """
    Calcualte the backward messages ~ beta values.
    
    return: beta
    """
    beta = np.ones( (observation_len, latent_len) )
    
    for t in range(observation_len-1):
        for j in range(latent_len):
            tmp_sum = np.multiply(beta[t+1], transition_matrix[j])
            beta[t][j] = np.sum( np.multiply(tmp_sum, emission_matrix[t+1]) )
    
    return beta

def calc_updated_matrix(alpha, beta, latent_len, observation_len):
    """
    Calcualte the gama and epsilon values in order to reproduce better transition and emission matrix.
    
    return: unary_matrix, transition_matrix, emission_matrix
    """
    gamma = np.zeros( (observation_len, latent_len) )
    epsilon = np.zeros( (observation_len-1, latent_len, latent_len) )
    
    for t in range(observation_len):
        gamma[t] = np.multiply(alpha[t], beta[t])
        # Normalization
        sum_gamma = np.sum(gamma[t])
        gamma[t] = np.divide(gamma[t], sum_gamma) 
        
    for t in range(observation_len - 1):   
        for i in range(latent_len):
            for j in range(latent_len):
                epsilon[t][i][j] = alpha[t][i]*transition_matrix[i][j]*beta[t+1][j]*emission_matrix[t+1][j]
        # Normalization
        sum_ep = np.sum(epsilon[t])
        epsilon[t] = np.divide(epsilon[t], sum_ep)
    
    # Update unary matrix
    unary_matrix = gamma[0]
    
    transition_matrix = np.zeros( (observation_len, latent_len) )
    emission_matrix = np.zeros( (latent_len, latent_len) )
    
    # Update transition matrix
    for i in range(latent_len):
        sum_gamma = np.sum(gamma.T[i])
        for j in range(latent_len):
            sum_ep = np.sum(epsilon.T[j][i])
            transition_matrix[i][j] = sum_ep/sum_gamma
    
    # Update emission matrix
    sum_gamma = [np.sum(gamma.T[i]) for i in range(latent_len)]    
    for i in range(latent_len):
        emission_matrix.T[i] = np.divide(gamma.T[i], sum_gamma[i])
    
    return unary_matrix, transition_matrix, emission_matrix